In [4]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2, ResNet50, EfficientNetB0
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve, average_precision_score
import seaborn as sns
from sklearn.utils.class_weight import compute_class_weight
!pip install keras-tuner
import keras_tuner as kt
import time
from datetime import datetime

# # Connect to Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Set correct paths to your dataset
base_dir = '/content/drive/MyDrive/MINI_PROJECT'
data_dir = os.path.join(base_dir, 'data')
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Create a directory for saving results
results_dir = os.path.join(base_dir, 'Results')
os.makedirs(results_dir, exist_ok=True)

# Create a log file for tracking results
log_file_path = os.path.join(results_dir, 'training_log.txt')
def log_message(message):
    with open(log_file_path, 'a') as log_file:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_file.write(f"[{timestamp}] {message}\n")
    print(message)

log_message(f"Starting skin disease classification project")
log_message(f"Train directory: {train_dir}")
log_message(f"Validation directory: {val_dir}")
log_message(f"Test directory: {test_dir}")

# Filter out hidden directories and get actual class names
train_classes = [d for d in os.listdir(train_dir) if not d.startswith('.') and os.path.isdir(os.path.join(train_dir, d))]
val_classes = [d for d in os.listdir(val_dir) if not d.startswith('.') and os.path.isdir(os.path.join(val_dir, d))]
test_classes = [d for d in os.listdir(test_dir) if not d.startswith('.') and os.path.isdir(os.path.join(test_dir, d))]

# Use the common classes across all directories
skin_classes = sorted(list(set(train_classes).intersection(set(val_classes)).intersection(set(test_classes))))
log_message(f"Classes used in model: {skin_classes}")
log_message(f"Number of classes: {len(skin_classes)}")

# Count images in each split
def count_images(directory, class_list):
    total = 0
    class_counts = {}
    for class_name in class_list:
        class_dir = os.path.join(directory, class_name)
        if os.path.isdir(class_dir):
            count = len([f for f in os.listdir(class_dir) if not f.startswith('.')])
            class_counts[class_name] = count
            total += count
    return total, class_counts

train_total, train_counts = count_images(train_dir, skin_classes)
val_total, val_counts = count_images(val_dir, skin_classes)
test_total, test_counts = count_images(test_dir, skin_classes)

log_message(f"Training images per class: {train_counts}")
log_message(f"Validation images per class: {val_counts}")
log_message(f"Test images per class: {test_counts}")

# Calculate class weights to handle imbalanced data
class_weights = {}
labels = []
for class_name in skin_classes:
    for _ in range(train_counts[class_name]):
        labels.append(skin_classes.index(class_name))

labels = np.array(labels)
weight = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
for i, class_name in enumerate(skin_classes):
    class_weights[i] = weight[i]

log_message(f"Class weights for training: {class_weights}")

# Visualize class distribution with better styling
plt.figure(figsize=(12, 6))
train_bars = plt.bar(train_counts.keys(), train_counts.values(), alpha=0.7, label='Training')
plt.bar(val_counts.keys(), [val_counts[k] for k in train_counts.keys()], alpha=0.7, bottom=[train_counts[k] for k in train_counts.keys()], label='Validation')
plt.xticks(rotation=45, ha='right')
plt.title('Class Distribution in Dataset', fontsize=16)
plt.xlabel('Class', fontsize=14)
plt.ylabel('Number of Images', fontsize=14)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'class_distribution.png'), dpi=300)
plt.close()

# Set image size and batch size
img_height, img_width = 224, 224
batch_size = 32

# Define improved data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,  # For skin images, vertical flip might not make sense
    fill_mode='nearest',
    brightness_range=[0.8, 1.2]  # Add brightness variation
)

# Just rescaling for validation and test
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    classes=skin_classes
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=skin_classes
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    classes=skin_classes
)

# Get class indices and prepare for later evaluation
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())
num_classes = len(class_names)
log_message(f"Class indices: {class_indices}")

# Show some sample images with improved visualization
def show_batch(image_batch, label_batch, save_path=None):
    plt.figure(figsize=(12, 12))
    for i in range(min(16, len(image_batch))):
        ax = plt.subplot(4, 4, i + 1)
        plt.imshow(image_batch[i])
        class_idx = np.argmax(label_batch[i])
        plt.title(f"{class_names[class_idx]}", fontsize=12)
        plt.axis("off")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
        plt.close()
    else:
        plt.show()

# Get a batch of images and save to results folder
images, labels = next(train_generator)
show_batch(images, labels, os.path.join(results_dir, 'sample_training_images.png'))

# Define a function to build model for hyperparameter tuning
def build_model(hp):
    # Choose a base model
    base_model_choice = hp.Choice('base_model', ['MobileNetV2', 'ResNet50', 'EfficientNetB0'])

    if base_model_choice == 'MobileNetV2':
        base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    elif base_model_choice == 'ResNet50':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    else:  # EfficientNetB0
        base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

    # Freeze or unfreeze base model
    base_model.trainable = hp.Boolean('trainable_base', default=False)

    # Create model
    model = Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        Dense(
            units=hp.Int('dense_units', min_value=128, max_value=512, step=64),
            activation=hp.Choice('dense_activation', ['relu', 'selu'])
        ),
        BatchNormalization(),
        Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile model
    model.compile(
        optimizer=Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-3, sampling='log')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy',
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall'),
                tf.keras.metrics.AUC(name='auc')]
    )

    return model

# Implement hyperparameter tuning
log_message("Starting hyperparameter tuning...")
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=15,
    factor=3,
    directory=os.path.join(results_dir, 'tuner'),
    project_name='skin_disease_classifier'
)

# Define early stopping callback for tuning
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Calculate steps correctly
steps_per_epoch = train_generator.samples // batch_size
validation_steps = validation_generator.samples // batch_size
steps_per_epoch = max(1, steps_per_epoch)
validation_steps = max(1, validation_steps)

log_message(f"Steps per epoch: {steps_per_epoch}")
log_message(f"Validation steps: {validation_steps}")

# Start the search
tuner.search(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[early_stopping],
    class_weight=class_weights
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
log_message(f"Best hyperparameters: {best_hps.values}")

# Build the best model
best_model = tuner.hypermodel.build(best_hps)
log_message("Best model architecture:")
best_model.summary(print_fn=lambda x: log_message(x))

# Set up callbacks for final training
model_checkpoint_path = os.path.join(results_dir, 'best_model.h5')
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        filepath=model_checkpoint_path,
        save_best_only=True,
        monitor='val_accuracy',
        mode='max'
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

# Train the best model for more epochs
log_message("Training the best model...")
start_time = time.time()
history = best_model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=callbacks,
    class_weight=class_weights
)
training_time = time.time() - start_time
log_message(f"Training complete in {training_time:.2f} seconds")

# Plot training history with improved visualization
def plot_training_history(history, save_path_prefix):
    metrics = ['accuracy', 'loss', 'precision', 'recall', 'auc']
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        if i < len(axes) and f'{metric}' in history.history:
            ax = axes[i]
            ax.plot(history.history[f'{metric}'], label=f'Training {metric}')
            ax.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
            ax.set_title(f'{metric.capitalize()} over Epochs', fontsize=14)
            ax.set_xlabel('Epoch', fontsize=12)
            ax.set_ylabel(metric.capitalize(), fontsize=12)
            ax.legend()
            ax.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(f"{save_path_prefix}_metrics.png", dpi=300)
    plt.close()

    # Create a separate learning curve plot
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig(f"{save_path_prefix}_learning_curve.png", dpi=300)
    plt.close()

plot_training_history(history, os.path.join(results_dir, 'training_history'))

# Load the best model (the one saved during training)
best_model = load_model(model_checkpoint_path)

# Comprehensive evaluation on test set
test_steps = max(1, test_generator.samples // batch_size)
log_message("Evaluating model on test set...")
test_loss, test_acc, test_precision, test_recall, test_auc = best_model.evaluate(test_generator, steps=test_steps)
log_message(f"Test accuracy: {test_acc:.4f}")
log_message(f"Test loss: {test_loss:.4f}")
log_message(f"Test precision: {test_precision:.4f}")
log_message(f"Test recall: {test_recall:.4f}")
log_message(f"Test AUC: {test_auc:.4f}")
log_message(f"F1 Score (approximated): {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

# Make predictions on test set with appropriate resetting
test_generator.reset()
y_pred_probs = best_model.predict(test_generator, steps=test_steps)
y_pred_classes = np.argmax(y_pred_probs, axis=1)

# Get true labels (ensuring we have the correct number)
y_true = test_generator.classes[:len(y_pred_classes)]

# Generate detailed classification report
class_report = classification_report(y_true, y_pred_classes, target_names=class_names, output_dict=True)
log_message("Classification Report:\n" + classification_report(y_true, y_pred_classes, target_names=class_names))

# Save classification report as CSV
class_report_df = pd.DataFrame(class_report).transpose()
class_report_df.to_csv(os.path.join(results_dir, 'classification_report.csv'))

# Plot confusion matrix with improved styling
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_true, y_pred_classes)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
sns.heatmap(cm_normalized, annot=cm, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names, annot_kws={"size": 12})
plt.title('Confusion Matrix', fontsize=16)
plt.ylabel('True Label', fontsize=14)
plt.xlabel('Predicted Label', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'confusion_matrix.png'), dpi=300)
plt.close()

# Plot ROC curves
plt.figure(figsize=(12, 10))
for i, class_name in enumerate(class_names):
    # Binarize the output for current class
    y_true_bin = (y_true == i).astype(int)
    y_score = y_pred_probs[:, i]

    # Compute ROC
    fpr, tpr, _ = roc_curve(y_true_bin, y_score)
    roc_auc = auc(fpr, tpr)

    # Plot ROC curve
    plt.plot(fpr, tpr, lw=2, label=f'{class_name} (AUC = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Curves for Each Class', fontsize=16)
plt.legend(loc="lower right")
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'roc_curves.png'), dpi=300)
plt.close()

# Plot Precision-Recall curves
plt.figure(figsize=(12, 10))
for i, class_name in enumerate(class_names):
    # Binarize the output for current class
    y_true_bin = (y_true == i).astype(int)
    y_score = y_pred_probs[:, i]

    # Compute PR curve
    precision, recall, _ = precision_recall_curve(y_true_bin, y_score)
    avg_precision = average_precision_score(y_true_bin, y_score)

    # Plot PR curve
    plt.plot(recall, precision, lw=2, label=f'{class_name} (AP = {avg_precision:.2f})')

plt.xlabel('Recall', fontsize=14)
plt.ylabel('Precision', fontsize=14)
plt.title('Precision-Recall Curves for Each Class', fontsize=16)
plt.legend(loc="best")
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(results_dir, 'precision_recall_curves.png'), dpi=300)
plt.close()

# Create a function to visualize correctly and incorrectly classified samples
def visualize_predictions(model, generator, class_names, num_samples=4, save_path=None):
    # Get a batch of test images
    generator.reset()
    batch_images, batch_labels = next(generator)

    # Make predictions
    predictions = model.predict(batch_images)

    # Create a figure with two rows: correct and incorrect predictions
    fig, axes = plt.subplots(2, num_samples, figsize=(16, 8))

    # Find correct and incorrect predictions
    pred_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_labels, axis=1)

    correct_indices = np.where(pred_classes == true_classes)[0]
    incorrect_indices = np.where(pred_classes != true_classes)[0]

    # Plot correct predictions
    correct_title = "Correctly Classified Examples"
    plt.figtext(0.5, 0.95, correct_title, ha='center', va='center', fontsize=16, fontweight='bold')

    for i in range(num_samples):
        if i < len(correct_indices):
            idx = correct_indices[i]
            ax = axes[0, i]
            ax.imshow(batch_images[idx])
            true_class = class_names[true_classes[idx]]
            pred_class = class_names[pred_classes[idx]]
            pred_prob = np.max(predictions[idx]) * 100
            ax.set_title(f"True: {true_class}\nPred: {pred_class}\nConf: {pred_prob:.1f}%", fontsize=10)
            ax.axis('off')
        else:
            axes[0, i].axis('off')

    # Plot incorrect predictions
    incorrect_title = "Misclassified Examples"
    plt.figtext(0.5, 0.48, incorrect_title, ha='center', va='center', fontsize=16, fontweight='bold')

    for i in range(num_samples):
        if i < len(incorrect_indices):
            idx = incorrect_indices[i]
            ax = axes[1, i]
            ax.imshow(batch_images[idx])
            true_class = class_names[true_classes[idx]]
            pred_class = class_names[pred_classes[idx]]
            pred_prob = predictions[idx][pred_classes[idx]] * 100
            ax.set_title(f"True: {true_class}\nPred: {pred_class}\nConf: {pred_prob:.1f}%", fontsize=10)
            ax.axis('off')
        else:
            axes[1, i].axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    if save_path:
        plt.savefig(save_path, dpi=300)
        plt.close()
    else:
        plt.show()

# Visualize model predictions
visualize_predictions(best_model, test_generator, class_names,
                     save_path=os.path.join(results_dir, 'prediction_examples.png'))

# Create an ensemble model (optional)
def create_ensemble():
    log_message("Creating ensemble model...")
    # Define different base models for ensemble
    base_models = ['MobileNetV2', 'ResNet50', 'EfficientNetB0']
    ensemble_models = []

    for i, base_name in enumerate(base_models):
        if base_name == 'MobileNetV2':
            base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
        elif base_name == 'ResNet50':
            base = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
        else:  # EfficientNetB0
            base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

        # Create model
        model = Sequential([
            base,
            tf.keras.layers.GlobalAveragePooling2D(),
            Dense(256, activation='relu'),
            BatchNormalization(),
            Dropout(0.3),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.0001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        # Save path for this ensemble member
        model_path = os.path.join(results_dir, f'ensemble_model_{i}.h5')

        # Set up callbacks
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
            ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_accuracy'),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
        ]

        # Train model
        log_message(f"Training ensemble member {i+1} with {base_name}...")
        model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            epochs=15,  # Fewer epochs for ensemble members
            validation_data=validation_generator,
            validation_steps=validation_steps,
            callbacks=callbacks,
            class_weight=class_weights
        )

        # Load the best version of this model
        ensemble_models.append(load_model(model_path))

    return ensemble_models

# Optionally create and evaluate ensemble
if train_total > 500:  # Only create ensemble if enough training data
    ensemble_models = create_ensemble()

    # Ensemble prediction function
    def ensemble_predict(models, generator, steps):
        generator.reset()
        predictions = None

        # Get predictions from each model
        for model in models:
            model_preds = model.predict(generator, steps=steps)
            if predictions is None:
                predictions = model_preds
            else:
                predictions += model_preds

        # Average predictions
        predictions /= len(models)
        return predictions

    # Evaluate ensemble
    log_message("Evaluating ensemble model...")
    ensemble_preds = ensemble_predict(ensemble_models, test_generator, test_steps)
    ensemble_classes = np.argmax(ensemble_preds, axis=1)

    # Calculate accuracy
    ensemble_acc = np.mean(ensemble_classes == y_true)
    log_message(f"Ensemble accuracy: {ensemble_acc:.4f}")

    # Generate classification report for ensemble
    log_message("Ensemble Classification Report:\n" +
                classification_report(y_true, ensemble_classes, target_names=class_names))

# Function to predict on a single image (for deployment)
def predict_image(model, image_path, class_names):
    from tensorflow.keras.preprocessing import image

    # Load and preprocess the image
    img = image.load_img(image_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0

    # Make prediction
    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction, axis=1)[0]

    # Print results
    print(f"Predicted class: {class_names[predicted_class]}")
    for i, class_name in enumerate(class_names):
        print(f"{class_name}: {prediction[0][i]:.4f}")

    # Display the image with prediction
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Predicted: {class_names[predicted_class]}")
    plt.axis('off')
    plt.show()

    return class_names[predicted_class], prediction[0]

# Save model architecture and summary as text file
with open(os.path.join(results_dir, 'model_summary.txt'), 'w') as f:
    # Redirect summary to file
    best_model.summary(print_fn=lambda x: f.write(x + '\n'))

    # Add model compilation details
    f.write("\nModel Compilation Details:\n")
    f.write(f"Optimizer: Adam\n")
    f.write(f"Learning Rate: {best_hps.values['learning_rate']}\n")
    f.write(f"Loss Function: categorical_crossentropy\n")
    f.write(f"Metrics: accuracy, precision, recall, AUC\n")

    # Add final performance metrics
    f.write("\nFinal Performance Metrics:\n")
    f.write(f"Test Accuracy: {test_acc:.4f}\n")
    f.write(f"Test Loss: {test_loss:.4f}\n")
    f.write(f"Test Precision: {test_precision:.4f}\n")
    f.write(f"Test Recall: {test_recall:.4f}\n")
    f.write(f"Test AUC: {test_auc:.4f}\n")
    f.write(f"F1 Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}\n")

# Save the final model
model_save_path = os.path.join(results_dir, 'final_skin_disease_model.h5')
best_model.save(model_save_path)
log_message(f"Final model saved to {model_save_path}")

# Create a JSON file with class mapping for deployment
import json
class_mapping = {i: class_name for i, class_name in enumerate(class_names)}
with open(os.path.join(results_dir, 'class_mapping.json'), 'w') as f:
    json.dump(class_mapping, f)

log_message("Model training and evaluation complete. All results saved to the Results directory.")

Trial 30 Complete [00h 43m 17s]
val_accuracy: 0.46875

Best val_accuracy So Far: 0.65625
Total elapsed time: 05h 31m 26s
Best hyperparameters: {'base_model': 'MobileNetV2', 'trainable_base': False, 'dense_units': 512, 'dense_activation': 'relu', 'dropout_rate': 0.4, 'learning_rate': 0.00014002116550133282, 'tuner/epochs': 15, 'tuner/initial_epoch': 5, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
Best model architecture:


Model: "sequential_1"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │

33/33 ━━━━━━━━━━━━━━━━━━━━ 84s 2s/step - accuracy: 0.2393 - auc: 0.5587 - loss: 2.2610 - precision: 0.2277 - recall: 0.1529 - val_accuracy: 0.3839 - val_auc: 0.7171 - val_loss: 1.4277 - val_precision: 0.5484 - val_recall: 0.1518 - learning_rate: 1.4002e-04
Epoch 2/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 20s 573ms/step - accuracy: 0.3125 - auc: 0.6398 - loss: 1.9980 - precision: 0.3182 - recall: 0.2188 - val_accuracy: 0.3705 - val_auc: 0.7190 - val_loss: 1.4222 - val_precision: 0.5806 - val_recall: 0.1607 - learning_rate: 1.4002e-04
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4993 - auc: 0.7928 - loss: 1.3877 - precision: 0.5411 - recall: 0.4158

33/33 ━━━━━━━━━━━━━━━━━━━━ 73s 2s/step - accuracy: 0.4994 - auc: 0.7931 - loss: 1.3870 - precision: 0.5412 - recall: 0.4159 - val_accuracy: 0.4821 - val_auc: 0.7893 - val_loss: 1.2570 - val_precision: 0.6768 - val_recall: 0.2991 - learning_rate: 1.4002e-04
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 18s 523ms/step - accuracy: 0.3125 - auc: 0.6819 - loss: 1.9448 - precision: 0.3200 - recall: 0.2500 - val_accuracy: 0.4777 - val_auc: 0.7890 - val_loss: 1.2593 - val_precision: 0.6735 - val_recall: 0.2946 - learning_rate: 1.4002e-04
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5472 - auc: 0.8198 - loss: 1.2850 - precision: 0.5922 - recall: 0.4711

33/33 ━━━━━━━━━━━━━━━━━━━━ 145s 3s/step - accuracy: 0.5475 - auc: 0.8203 - loss: 1.2829 - precision: 0.5926 - recall: 0.4712 - val_accuracy: 0.5848 - val_auc: 0.8244 - val_loss: 1.1643 - val_precision: 0.7391 - val_recall: 0.3795 - learning_rate: 1.4002e-04
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 17s 504ms/step - accuracy: 0.5000 - auc: 0.8746 - loss: 1.0150 - precision: 0.6400 - recall: 0.5000 - val_accuracy: 0.5804 - val_auc: 0.8261 - val_loss: 1.1595 - val_precision: 0.7414 - val_recall: 0.3839 - learning_rate: 1.4002e-04
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5862 - auc: 0.8672 - loss: 1.0686 - precision: 0.6448 - recall: 0.5178

...

Training ensemble member 3 with EfficientNetB0...
Epoch 1/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 11s/step - accuracy: 0.2346 - loss: 2.1966 

33/33 ━━━━━━━━━━━━━━━━━━━━ 446s 12s/step - accuracy: 0.2361 - loss: 2.1912 - val_accuracy: 0.2009 - val_loss: 1.6350 - learning_rate: 1.0000e-04
Epoch 2/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 40s 876ms/step - accuracy: 0.5938 - loss: 1.2936 - val_accuracy: 0.2009 - val_loss: 1.6345 - learning_rate: 1.0000e-04
Epoch 3/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 442s 13s/step - accuracy: 0.4693 - loss: 1.4417 - val_accuracy: 0.1964 - val_loss: 1.6481 - learning_rate: 1.0000e-04
Epoch 4/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 998ms/step - accuracy: 0.5312 - loss: 1.2467 - val_accuracy: 0.1964 - val_loss: 1.6505 - learning_rate: 1.0000e-04
Epoch 5/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 411s 11s/step - accuracy: 0.5995 - loss: 1.0530 - val_accuracy: 0.1964 - val_loss: 1.7091 - learning_rate: 1.0000e-04
Epoch 6/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 42s 964ms/step - accuracy: 0.7500 - loss: 0.8389 - val_accuracy: 0.1964 - val_loss: 1.7110 - learning_rate: 5.0000e-05
Epoch 7/15
33/33 ━━━━━━━━━━━━━━━━━━━━ 442s 13s/step - accuracy: 0.6306 -

Evaluating ensemble model...
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 43s 6s/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step
Ensemble accuracy: 0.3170
Ensemble Classification Report:
              precision    recall  f1-score   support

        acne       0.25      0.04      0.08        45
       eksim       0.38      0.13      0.20        45
      herpes       0.00      0.00      0.00        45
        panu       0.30      0.78      0.44        45
     rosacea       0.33      0.64      0.43        44

    accuracy                           0.32       224
   macro avg       0.25      0.32      0.23       224
weighted avg       0.25      0.32      0.23       224



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Final model saved to /content/drive/MyDrive/MINI_PROJECT/Results/final_skin_disease_model.h5
Model training and evaluation complete. All results saved to the Results directory.
